#Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import stats

#Preliminary Analysis

In [ ]:
train_df = pd.read_csv('./train.csv')
train_df2 = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
train_df.head()

NameError: name 'train_df' is not defined

In [ ]:

train_df.info()

train_df.describe().T

## chooing to drop nan or not should come by looking at the scatter plots/value counts hist of the data
#for example nan in alley means no alley which is information not to throw away. make it a 'noAlley' category

In [ ]:
#MISSING VALUES

missing = pd.DataFrame(train_df.isnull().sum().sort_values(ascending=False))
missing.columns = ['count']
missing = missing.loc[(missing!=0).any(axis=1)]
missing['percentage'] = missing[0:]/1460
missing.style.background_gradient()


In [ ]:
# Getting numeric and non numeric columns

non_numeric_cols = list(train_df.dtypes[train_df.dtypes == 'object'].index)  # Non-numeric
numeric_cols = list(train_df.dtypes[train_df.dtypes != 'object'].index)     # Numeric

In [ ]:
# IMPUTATION

# Numeric features:
df['A'] = df['A'].replace([np.inf, -np.inf], np.nan)

df['A'].fillna(method='ffill', inplace = True)
df['A'].fillna(0, inplace = True)
df['A_interpolated_linear'] = df['A'].interpolate(method='linear')
df['A_interpolated_quadratic'] = df['A'].interpolate(method='polynomial', order=2)


# Categorical features:
imputer = SimpleImputer(strategy='constant', fill_value='Do_not_have_this_feature')
df[['feature_1', 'feature_2']] = imputer.fit_transform(df[['feature_1', 'feature_2']])

#Ordinal features
imputer = SimpleImputer(strategy='median')
df[['feature_1', 'feature_2']] = imputer.fit_transform(df[['feature_1', 'feature_2']])

###Time series snippets

In [ ]:
# if day 1, day 2 etc are columns in the dataframe instead of rows and we need to do time series regression, then use this

##melt data

df_melt = pd.melt(df, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')

#this will create a dataframe with column called 'd' where all the columns not in id_vars will be in each row and the value_name is the target variable


In [ ]:
# 1. Decompose Time Series
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Decompose the time series into trend, seasonal, and residual components
result = seasonal_decompose(df['value_column'], model='additive', period=12)  # Adjust period for data's seasonality

# Plot decomposed components
result.plot()
plt.show()

# Extract components into the dataset
df['trend'] = result.trend
df['seasonal'] = result.seasonal
df['residual'] = result.resid

# 2. Check for Stationarity (e.g., using Augmented Dickey-Fuller test)
from statsmodels.tsa.stattools import adfuller

adf_test = adfuller(df['value_column'].dropna())
print("ADF Statistic:", adf_test[0])
print("p-value:", adf_test[1])  # p < 0.05 indicates stationarity

# 3. Plot ACF and PACF
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure()
plot_acf(df['value_column'].dropna(), lags=20)  # Adjust lags based on data
plt.show()

plt.figure()
plot_pacf(df['value_column'].dropna(), lags=20)  # Adjust lags based on data
plt.show()

# Exploratory

In [ ]:
#For classification, also do class imbalance analysis in the beginning

from imblearn.over_sampling import SMOTE

# Define SMOTE oversampler
smote = SMOTE(random_state=42)

# Apply SMOTE
X_resampled, y_resampled = smote.fit_resample(data.drop('target', axis=1), data['target'])

# Check new class distribution
print("New Class Distribution:\n", pd.Series(y_resampled).value_counts())

In [ ]:
#String manipulation
data['Cabin'] = data['Cabin'].apply(lambda x: str(x)[0] if pd.notnull(x) else "Not Defined")

In [ ]:
# PLOTTING

# 1. Scatter Plot

fig, ax = plt.subplots(12, 3, figsize=(23, 60))
for var, subplot in zip(numerical_features, ax.flatten()):
    sns.scatterplot(x=var, y='SalePrice',  data=train_df, ax=subplot, hue = 'SalePrice')

# 2. Pair Plot
sns.pairplot(df)
plt.title('Pairplot of Variables')
plt.show()

# 3. Categorical feature plotting
sns.catplot(data=train_df, x="Alley", y="SalePrice", kind="box")

In [ ]:
# CORRELATION ANALYSIS

plt.figure(figsize=(20,20))
corr_mat = train_df[numerical_features].corr()
sns.heatmap(corr_mat, cmap='coolwarm', annot=True, fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# FEATURES HISTOGRAM

for var in numerical_features:
    plt.figure(figsize=(8, 4))  # Set figure size
    plt.hist(train_df[var].dropna(), bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title(f"Histogram of {var}")
    plt.xlabel(var)
    plt.ylabel("Frequency")
    plt.show()


#Instead of histogram, can also use distplot():

sns.distplot(data['Fare'], kde=True, bins=80, color='green')
plt.title('Distribution Plot of a Fare of Passengers')
plt.show()

In [ ]:
# MUTUAL INFORMATION
from sklearn.feature_selection import mutual_info_classif
mutual_info_func = mutual_info_classif if is_classification else mutual_info_regression

# Compute mutual information
mutual_info = mutual_info_func(X, y, random_state=42)

# Organize results into a DataFrame
mutual_info_df = pd.DataFrame({
    'Feature': X.columns,
    'Mutual Information': mutual_info
}).sort_values(by='Mutual Information', ascending=False)

In [ ]:
# CHECKING NORMAL DISTRIBUTION

# Plotting the distribution
sns.histplot(data, kde=True, stat="density", linewidth=0)
plt.title('Distribution of value_column')
plt.xlabel('Value')
plt.ylabel('Density')

# Overlay a normal distribution
mu, std = norm.fit(data)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.show()

In [ ]:
# OUTLIER DETECTION

## scatter plot feature vs target and manually drop like
plt.scatter(x='LotArea', y='SalePrice', data=train_df)
plt.show()

##robust z score
from statsmodels.robust.scale import mad
median = train_df[numeric_cols].median()
mad_value = mad(train_df[numeric_cols])
robust_z_score = 0.6745 * ((train_df[numeric_cols] - median) / mad_value)
threshold = 3
df_robust_z = train_df[numeric_cols][~(np.abs(robust_z_score) > threshold).any(axis=1)]

###regular z-score
stats.zscore(train_df['LotArea']).sort_values().tail(10)

###for categorical
plt.scatter(x='OverallQual', y='SalePrice', data=train_df)
plt.show()
# train_df.query('OverallCond == 10 & SalePrice < 200000') remove like this

##IQR
# Q1 = df.quantile(0.25)
# Q3 = df.quantile(0.75)
# IQR = Q3 - Q1
# df_iqr = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


##Percentile

lower_bound = df['feature'].quantile(0.01)
upper_bound = df['feature'].quantile(0.99)
outliers = df[(df['feature'] < lower_bound) | (df['feature'] > upper_bound)]

In [ ]:
#Dropping particular indices

df = df.drop(list_of_indices)
df_dropped = df.drop(columns=['Age', 'Score'])

###Categorical Featuress

In [ ]:
### See how target variable 1 or 0 etc is ditributed among the categorical and other pivot tables to help check trends/importance/distributions

# compare survival rate across Age, SibSp, Parch, and Fare -> see how higher fare survived more (numeric)
pd.pivot_table(train_df2, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

# compare survival across Pclass, Sex, Embarked -> see how female/male works (non-numeric)

print(pd.pivot_table(train_df2, index = 'Survived', columns = 'Pclass', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(train_df2, index = 'Survived', columns = 'Sex', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(train_df2, index = 'Survived', columns = 'Embarked', values = 'Ticket' ,aggfunc ='count'))

In [ ]:
# This simply plots bar graph where x-axis is the unique categories and y axis is the value counts

for i in ['Survived','Pclass','Sex','Ticket','Cabin','Embarked']: ##categorical
    sns.barplot(train_df2[i].value_counts().index,train_df2[i].value_counts()).set_title(i)
    plt.show()

#Cabin and ticket graphs are very messy. This is an area where we may want to do some feature engineering!

# Feature Engineering

In [ ]:
#Log transform: Look at historgram and then log histogram of target variables (use np.log1p)

In [ ]:
#One hot encoding vs ordinal encoding for categorical features: see scatterplot of the categorical features vs y

#One Hot Encoding
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False, drop=None)
# Transform the 'Color' column
encoded = one_hot_encoder.fit_transform(data[['Color']])
# Combine with original DataFrame
data_one_hot = pd.DataFrame(encoded, columns=one_hot_encoder.get_feature_names_out(['Color']))
data = pd.concat([data, data_one_hot], axis=1)



## Ordinal Encoding
# Define the order for encoding
education_order = ['High School', 'Bachelor', 'Master', 'PhD']
ordinal_encoder = OrdinalEncoder(categories=[education_order])

data['Education_Encoded'] = ordinal_encoder.fit_transform(data[['Education']])



# Target encoding for categorical variables

from category_encoders import TargetEncoder


encoder = TargetEncoder()
df['Category_encoded'] = encoder.fit_transform(df['Category'], df['Target'])
print(df)


#Do it manually
category_median = df.groupby('Category')['Target'].median()
df['Category_encoded'] = df['Category'].map(category_median)


In [ ]:
#Proper Data

def preprocess(data):

    imputer1 = SimpleImputer(strategy='mean')
    data[continuous_numerical_features] = imputer1.fit_transform(data[continuous_numerical_features])

    imputer2 = SimpleImputer(strategy='constant', fill_value='Do_not_have_this_feature')
    data = data.fillna(data[nominal_features+ ordinal_features].mode().iloc[0])

    #One Hot Encoding
    one_hot_encoder = OneHotEncoder(handle_unknown='ignore', drop=None)
    encoded = one_hot_encoder.fit_transform(data[nominal_features])
    encoded_dense = encoded.toarray()
    data_one_hot = pd.DataFrame(encoded_dense, columns=one_hot_encoder.get_feature_names_out(nominal_features), index=data.index)
    data = pd.concat([data, data_one_hot], axis=1)
    data = data.drop(columns=nominal_features, axis=1)

    # Label Encoding for specific features
    label_encoders = {}
    for feature in label_encoding_features:
        label_encoder = LabelEncoder()
        data[feature] = label_encoder.fit_transform(data[feature])
        label_encoders[feature] = label_encoder  # Save the encoder for potential inverse transformations later

    #Ordinal encoding
    ordinal_encoder = OrdinalEncoder()
    data[ordinal_features] = ordinal_encoder.fit_transform(data[ordinal_features])

    data = data[continuous_numerical_features + data_one_hot.columns.tolist() + ordinal_features + ['SalePrice']]

    return data, imputer1, imputer2, one_hot_encoder, ordinal_encoder

def process_test(data, imputer1, one_hot_encoder, ordinal_encoder ):
    data[continuous_numerical_features] = imputer1.transform(data[continuous_numerical_features])
    data = data.fillna(data[nominal_features+ ordinal_features].mode().iloc[0])
    encoded = one_hot_encoder.transform(data[nominal_features])
    encoded_dense = encoded.toarray()
    data_one_hot = pd.DataFrame(encoded_dense, columns=one_hot_encoder.get_feature_names_out(nominal_features), index=data.index)
    data = pd.concat([data, data_one_hot], axis=1)
    data = data.drop(columns=nominal_features, axis=1)
    data[ordinal_features] = ordinal_encoder.transform(data[ordinal_features])
    data = data[continuous_numerical_features + data_one_hot.columns.tolist() + ordinal_features + ['SalePrice']]
    return data

In [ ]:
# Split in train and test data (completely unseen)
train_df, test_df = train_test_split(full_dataset, test_size=0.2, random_state=42)

# Make X_train and y_train from processed data
train_df_proc, imputer1, imputer2, one_hot_encoder, ordinal_encoder = preprocess(train_df)
test_df_proc = process_test(test_df, imputer1, one_hot_encoder, ordinal_encoder)

## Time-Series

#### Rolling mean based on time

In [ ]:
# Set 'timestamp' as the index (optional, depends on your use case)
df.set_index('timestamp', inplace=True)

# Calculate rolling mean with a time-based window of 3 hours
df['rolling_mean'] = df['value'].rolling('3H').mean()

                     value  rolling_mean
timestamp                               
2023-01-01 00:00:00   10.0          10.0
2023-01-01 01:00:00   20.0          15.0
2023-01-01 02:00:00    NaN          15.0
2023-01-01 03:00:00   40.0          30.0
2023-01-01 04:00:00   50.0          45.0
2023-01-01 05:00:00   60.0          50.0
2023-01-01 06:00:00   70.0          60.0
2023-01-01 07:00:00   80.0          70.0
2023-01-01 08:00:00   90.0          80.0
2023-01-01 09:00:00  100.0          90.0


<ipython-input-2-33f00e66a605>:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'timestamp': pd.date_range(start='2023-01-01', periods=10, freq='H'),
<ipython-input-2-33f00e66a605>:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['rolling_mean'] = df['value'].rolling('3H').mean()


#### Other time series snippets

In [ ]:
## Time Series Data!!

###LAG BASED FEATURES - 28 DAY SHIFT ETC
### ROLLING MEAN/STD FOR DIFFERENT PERIODS
###MOMENTUM LIKE FEATURES EWMA ETC
##DIFFERENT MODELS PER DAY/PER STORE
## GROUP BY PER DAY/PER STORE ETC
##DAYS SINCE LAST SOLD
##LAST SOLD PRICE
##PROMOTION CYCLES, DAYS SINCE PROMOTIONS
## BY DAYS OF WEEK ETC
##Differencing for trends versus means or difference vs last time or something

melt_train["lag_sales_1"] = melt_train.groupby("Product_Code")['Sales'].shift(1)
melt_train["diff_sales_1"] = melt_train.groupby("Product_Code")['Sales'].diff(1)
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)


df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['quarter'] = df.index.quarter
df['month'] = df.index.month
df['year'] = df.index.year
df['dayofyear'] = df.index.dayofyear
df['dayofmonth'] = df.index.day
df['weekofyear'] = df.index.isocalendar().week


def add_lags(df):
    target_map = df['PJME_MW'].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    return df

#Introduce lags
lags = [1,2,3,6,12,24,36]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)


# Calculate log returns and pct returns
for col in data.columns:
    # Log returns
    data[f'{col}_log_return'] = np.log(data[col] / data[col].shift(1))
    # Percentage returns
    data[f'{col}_pct_return'] = data[col].pct_change()

# WARNING: Has future info
df['iteam_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)


df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.expanding(2).mean())

In [ ]:
#### Time Series Cross Validation: roll forward

tss = TimeSeriesSplit(n_splits=5, test_size=24*365*1, gap=24) #Can also add a max_train_size arg so that it becomes rolling after a while

df = df.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    train = create_features(train)
    test = create_features(test)

    FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month','year',
                'lag1','lag2','lag3']
    TARGET = 'PJME_MW'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

#Processing Pipeline

In [ ]:
#Pipelining

num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

ode_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ode', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

ohe_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

col_trans = ColumnTransformer(transformers=[
    ('num_p', num_pipeline, num_cols),
    ('ode_p', ode_pipeline, ode_cols),
    ('ohe_p', ohe_pipeline, ohe_cols),
    ],
    remainder='passthrough')

#remainder = drop if we want to drop


pipeline = Pipeline(steps = [
    ('preprocessing', col_trans)
])

X = train_df.drop('target_var')
y = train_df('target_var')

x_preprocessed = pipeline.fit_transform(X)

In [ ]:
# PIPELINE WITH GRIDSEARCH

# Sort by timestamp (important for time series data)
data = data.sort_values(by='Timestamp').reset_index(drop=True)

# Define features and target
X = data.drop(columns=['Target', 'Timestamp'])
y = data['Target']

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Define column groups
numerical_features = ['Numerical1', 'Numerical2']
categorical_features_onehot = ['Categorical']  # For one-hot encoding
categorical_features_target = ['Categorical']  # For target encoding

# Define preprocessing for each column group
numerical_transformer = StandardScaler()
onehot_transformer = OneHotEncoder(handle_unknown='ignore')
target_transformer = TargetEncoder()

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('onehot', onehot_transformer, categorical_features_onehot),
        ('target', target_transformer, categorical_features_target)
    ],
    remainder='drop'
)

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Define parameter grid for GridSearchCV
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10],
    'classifier__min_samples_split': [2, 5]
}

# TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# GridSearchCV with TimeSeriesSplit
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=tscv,            # TimeSeriesSplit ensures no look-ahead bias
    scoring='accuracy', # Metric to optimize
    n_jobs=-1,          # Use all available CPU cores
    verbose=1
)

# Perform grid search on the train set
grid_search.fit(X_train, y_train)

# Best parameters and cross-validation score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.2f}")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {test_accuracy:.2f}")


In [ ]:
# Define the number of principal components
n_components = 5

# Define the pipeline: Standardization -> PCA -> Regression
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),             # Standardize features
    ('pca', PCA(n_components=n_components)),  # Reduce dimensionality
    ('regressor', LinearRegression())         # Perform regression
])

# Define K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(pipeline, X, y, cv=kf, scoring='r2')  # Using R² as the metric

# Display the results
print(f"Cross-validated R² scores: {cv_scores}")
print(f"Mean R² score: {cv_scores.mean():.4f}")

In [ ]:
education_mapping = {
    'High School': 0,
    'Bachelor': 1,
    'Master': 2,
    'PhD': 3
}

satisfaction_mapping = {
    'Low': 0,
    'Medium': 1,
    'High': 2
}

# Apply mappings
data['Education_Encoded'] = data['Education'].map(education_mapping)
data['Satisfaction_Encoded'] = data['Satisfaction'].map(satisfaction_mapping)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_preprocessed, y, test_size=0.2, random_state=42)

# Models

In [ ]:
# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'ridge': Ridge(),
    'GradBoost':GradientBoostingRegressor(),
    'lgbm' : lgb.LGBMRegressor(),
}

param_grids = {
    'LinearRegression': {},

    'RandomForest': {
        'n_estimators': [100, 200],
        'max_depth': [None, 5, 10, 30],
        'min_samples_split': [2, 5, 10],
    },

    'XGBoost': {
        'learning_rate': [0.05, 0.1],
        'n_estimators': [200, 300],
        'max_depth': [3, 5],
        'min_child_weight': [1, 2],
        'gamma': [0, 0.1],
        'subsample': [0.8, 0.9],
        'colsample_bytree': [0.8, 0.9],
    },

    'ridge': {
        'alpha': [0.05, 0.1, 1, 5, 10],
        'solver': ['auto', 'svd', 'sparse_cg'],
    },

    'GradBoost': {
        'max_depth': [6, 10, 15],
        'n_estimators': [200, 300],
        'min_samples_leaf': [10, 25],
        'learning_rate': [0.01, 0.1],
        'max_features': ['sqrt', 'log2'],
    },

    'lgbm': {
        'boosting_type': ['gbdt', 'dart'],
        'num_leaves': [20, 30, 40],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids = {}
best_estimator = {}

for model_name, model in models.items():
    print(f'Training and tuning {model_name}...')
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_estimator[model_name] = grids[model_name].best_estimator_
    best_score = np.sqrt(-1 * grids[model_name].best_score_)

    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')


In [ ]:
## Voting Ensemble Regressors
vr = VotingRegressor([('lgbm', best_estimator['lgbm']),
                      ('GradBoost', best_estimator['GradBoost']),
                      ('ridge', best_estimator['ridge'])],
                    weights=[3,2,1]) ##hard and soft voting

vr.fit(X_train, y_train)

y_pred_vr = vr.predict(X_test)

In [ ]:
#Classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold

models = {
    'LogisticRegression': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'SVC': SVC(probability=True),
    'XGBoost': XGBClassifier(random_state=1),
    'RandomForest': RandomForestClassifier(random_state=1),
    'LightGBM': lgb.LGBMClassifier(random_state=1)

}

# Define the hyperparameter grids for each model
param_grids = {
    'LogisticRegression': {
        'max_iter': [1000],  # 1000 is usually enough; 2000 may be overkill
        'penalty': ['l1', 'l2'],
        'C': np.logspace(-3, 3, 7),  # Reduce range for efficiency; 7 values are sufficient
        'solver': ['liblinear'],  # Required for L1 and L2 penalties
    },

    'KNN': {
        'n_neighbors': [3, 5, 7],  # Focused range for efficient tuning
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto'],  # Simplify to 'auto' for general use
        'p': [1, 2],  # Manhattan (p=1) or Euclidean (p=2) distances
    },

    'SVC': {
        'kernel': ['rbf', 'linear', 'poly'],  # Consider the 3 main kernel types
        'C': [0.1, 1, 10, 100],  # Regularization parameter
        'gamma': ['scale', 'auto'],  # For 'rbf' and 'poly' kernels
        'degree': [2, 3],  # Relevant only for 'poly' kernel
    },

    'XGBoost': {
        'n_estimators': [100, 200],  # Reduced for efficiency
        'colsample_bytree': [0.8, 0.9],
        'max_depth': [3, 5],  # Common values for max depth
        'reg_alpha': [0.1, 1],  # L1 regularization
        'reg_lambda': [1, 2],  # L2 regularization
        'subsample': [0.8, 1.0],
        'learning_rate': [0.05, 0.1],  # Reduced to a reasonable range
        'gamma': [0, 0.1, 0.2],
        'min_child_weight': [1, 2],
    },

    'RandomForest': {
        'n_estimators': [100, 200, 300],  # Commonly used values
        'criterion': ['gini', 'entropy'],  # Impurity measures
        'bootstrap': [True],  # Standard setting for Random Forest
        'max_depth': [10, 20],  # Depth of the trees
        'max_features': ['auto', 'sqrt'],  # 'auto' = all features; 'sqrt' = sqrt of features
        'min_samples_leaf': [1, 2],  # Small leaf sizes for flexibility
        'min_samples_split': [2, 5],  # Standard splitting options
    },
     'LightGBM':  {
      'n_estimators': [100, 200],           # Small range for boosting rounds
      'max_depth': [-1, 5],                # No limit (-1) and a moderate value
      'learning_rate': [0.05, 0.1],        # Common learning rates
      'subsample': [0.8, 1.0],             # Slight row sampling
      'colsample_bytree': [0.8, 1.0],      # Slight feature sampling
      'num_leaves': [31, 50],              # Default (31) and slightly larger
      'reg_lambda': [0, 1.0],              # L2 regularization for overfitting
    }
}



# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids = {}
best_estimator = {}

for model_name, model in models.items():
    print(f'Training and tuning {model_name}...')
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='f1', verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_estimator[model_name] = grids[model_name].best_estimator_
    best_score = grids[model_name].best_score_

    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best F1-score for {model_name}: {best_score}\n')


In [ ]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier([('rfr', best_estimator['rfr']),
                      ('xgb', best_estimator['XGBoost']),
                      ('logistic', best_estimator['LogisticRegression'])],
                    weights=[2,3,1], voting = 'soft')
vc.fit(X_train, y_train)
y_pred_vc = vc.predict(X_test)
f1_score(y_test, y_pred_vc, average='weighted')



In [ ]:
def rolling_cross_validation(df, model, window_size, val_size, test_size, evaluate_model):
    dates = df.dates.unique()
    n = len(dates)

    # List to store validation scores
    val_scores = []

    # Start from window_size to ensure the training data can extend backward
    for i in range(window_size, n - val_size - test_size + 1, val_size):
        # Define training and validation windows
        dates_train = dates[i-window_size:i]  # Training window (past `window_size` dates)
        dates_val = dates[i:i+val_size]      # Validation window (next `val_size` dates)

        # Filter data for the respective sets
        train_data = df[df['dates'].isin(dates_train)]
        val_data = df[df['dates'].isin(dates_val)]

        # Train and evaluate the model
        model.fit(train_data.drop(columns=['target']), train_data['target'])
        val_predictions = model.predict(val_data.drop(columns=['target']))
        val_score = evaluate_model(val_predictions, val_data['target'])
        val_scores.append(val_score)

        print(f"Window ending at index {i}: Validation Score: {val_score}")

    # Compute the average validation score
    average_val_score = np.mean(val_scores)
    print(f"Average Validation Score across all splits: {average_val_score}")

    return average_val_score, val_scores



In [ ]:
def rolling_test_validation(df, model, window_size, val_size, test_size, evaluate_model):
    dates = df.dates.unique()
    n = len(dates)

    # List to store validation scores and predictions
    val_scores = []
    all_predictions = []

    # Start from window_size to ensure the training data can extend backward
    for i in range(n - test_size, n - val_size - 1, val_size):
        # Define training and validation windows
        dates_train = dates[i-window_size:i]  # Training window (past `window_size` dates)
        dates_val = dates[i:i+val_size]      # Validation window (next `val_size` dates)

        # Filter data for the respective sets
        train_data = df[df['dates'].isin(dates_train)]
        val_data = df[df['dates'].isin(dates_val)]

        # Train and evaluate the model
        model.fit(train_data.drop(columns=['target']), train_data['target'])
        val_predictions = model.predict(val_data.drop(columns=['target']))
        val_score = evaluate_model(val_predictions, val_data['target'])

        # Append validation score and predictions
        val_scores.append(val_score)
        all_predictions.extend(val_predictions)  # Flatten the predictions

        print(f"Window ending at index {i}: Validation Score: {val_score}")

    # Compute the average validation score
    average_val_score = np.mean(val_scores)
    print(f"Average Validation Score across all splits: {average_val_score}")

    return average_val_score, val_scores, all_predictions


In [ ]:
# TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=3)

# Train and tune the models
grids = {}
best_estimator = {}

for model_name, model in models.items():
    print(f'Training and tuning {model_name}...')
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=tscv, scoring='neg_mean_squared_error', verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_estimator[model_name] = grids[model_name].best_estimator_
    best_score = np.sqrt(-1 * grids[model_name].best_score_)

    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')


In [ ]:
#### Time Series Cross Validation

tss = TimeSeriesSplit(n_splits=5, test_size=24*365*1, gap=24)
df = df.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    train = create_features(train)
    test = create_features(test)

    FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month','year',
                'lag1','lag2','lag3']
    TARGET = 'PJME_MW'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

#Model Evaluation

In [ ]:
cm = confusion_matrix(y_test, y_pred_vc)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=vc.classes_, yticklabels=vc.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
